In [40]:
import pandas as pd
import os
from os.path import dirname


root_path = dirname(os.getcwd())

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs_repair/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

/home/sebdis/ProcessMining/Seph_outcome/SEPH_outcome
/home/sebdis/ProcessMining/Seph_outcome/SEPH_outcome/data/datasets/original/
/home/sebdis/ProcessMining/Seph_outcome/SEPH_outcome/data/datasets/processed/
/home/sebdis/ProcessMining/Seph_outcome/SEPH_outcome/data/datasets/graphs_repair/


In [2]:
dataset = "BPI12_DECLINED_COMPLETE"

In [3]:
raw_data = pd.read_csv(f"datasets/original/{dataset}.csv")
raw_data.head()


,Resource,lifecycle:transition,timesincemidnight,timesincelastevent,timesincecasestart,event_nr,month,weekday,hour,open_cases,time:timestamp,concept:name,case:AMOUNT_REQ,case:concept:name,case:label
0,0.0,SCHEDULE,941.0,6.113517,24061.486817,46.0,11.0,4.0,15.0,758.0,2011-11-18 15:41:40.707000+00:00,wwijzigencontractgegevensschedule,25000.0,181447,False
1,0.0,SCHEDULE,942.0,0.741917,24062.228733,47.0,11.0,4.0,15.0,758.0,2011-11-18 15:42:25.222000+00:00,wwijzigencontractgegevensschedule,25000.0,181447,False
2,0.0,START,899.0,11766.656217,12505.183217,13.0,11.0,3.0,14.0,737.0,2011-11-17 14:59:22.469000+00:00,wnabellenoffertesstart,5000.0,183277,False
3,0.0,COMPLETE,435.0,975.723167,13480.906383,14.0,11.0,4.0,7.0,788.0,2011-11-18 07:15:05.859000+00:00,wnabellenoffertescomplete,5000.0,183277,False
4,0.0,START,570.0,15.111433,61129.463333,29.0,12.0,2.0,9.0,646.0,2011-12-21 09:30:08.082000+00:00,wvaliderenaanvraagstart,12850.0,183280,False


In [39]:
tab_all = raw_data.rename(
    columns={"case:concept:name": "CaseID", "concept:name": "Activity"}
)

In [43]:
from datetime import datetime
import time

def translate_time(time_str):
    return datetime.fromisoformat(time_str).timestamp()

In [44]:
tab_all["time:timestamp"] = tab_all["time:timestamp"].apply(translate_time)

In [45]:
tab_all.head()

,Resource,lifecycle:transition,timesincemidnight,timesincelastevent,timesincecasestart,event_nr,month,weekday,hour,open_cases,time:timestamp,Activity,case:AMOUNT_REQ,CaseID,case:label
0,0.0,SCHEDULE,941.0,6.113517,24061.486817,46.0,11.0,4.0,15.0,758.0,1.321631e+09,wwijzigencontractgegevensschedule,25000.0,181447,False
1,0.0,SCHEDULE,942.0,0.741917,24062.228733,47.0,11.0,4.0,15.0,758.0,1.321631e+09,wwijzigencontractgegevensschedule,25000.0,181447,False
2,0.0,START,899.0,11766.656217,12505.183217,13.0,11.0,3.0,14.0,737.0,1.321542e+09,wnabellenoffertesstart,5000.0,183277,False
3,0.0,COMPLETE,435.0,975.723167,13480.906383,14.0,11.0,4.0,7.0,788.0,1.321601e+09,wnabellenoffertescomplete,5000.0,183277,False
4,0.0,START,570.0,15.111433,61129.463333,29.0,12.0,2.0,9.0,646.0,1.324460e+09,wvaliderenaanvraagstart,12850.0,183280,False


In [46]:
split_ratio = 2 / 3

first_act_tab = (
    tab_all.groupby("CaseID").first().sort_values("time:timestamp").reset_index()
)
first_act_tab = first_act_tab[
    ~first_act_tab.duplicated(subset=["CaseID", "Activity"], keep="first")
]
first_act_tab = first_act_tab.reset_index(drop=True)

list_train_valid_cases = list(
    first_act_tab[: int(split_ratio * len(first_act_tab))]["CaseID"].unique()
)

list_train_cases = list_train_valid_cases[: int(len(list_train_valid_cases) * 0.8)]
tab_train = tab_all[tab_all["CaseID"].isin(list_train_cases)].reset_index(drop=True)

list_valid_cases = list_train_valid_cases[int(len(list_train_valid_cases) * 0.8) :]
tab_valid = tab_all[tab_all["CaseID"].isin(list_valid_cases)].reset_index(drop=True)

list_test_cases = list(
    first_act_tab[int(split_ratio * len(first_act_tab)) :]["CaseID"].unique()
)
tab_test = tab_all[tab_all["CaseID"].isin(list_test_cases)].reset_index(drop=True)

In [47]:
tab_all.to_csv(data_dir_processed + f"{dataset}_processed_all.csv", index=False)

In [48]:
tab_train.to_csv(data_dir_processed+ f"{dataset}_processed_train.csv", index = False)

In [49]:
tab_valid.to_csv(data_dir_processed+f"{dataset}_processed_valid.csv", index = False)

In [50]:
tab_test.to_csv(data_dir_processed+ f"{dataset}_processed_test.csv", index = False)